In [1]:
# python
import os
import math
import csv

# random
import random
#data analysis libraries
import numpy as np
import pandas as pd

# machine learning
import sklearn

# deep learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import pytorch_lightning as pl

#d2l
import d2l.torch as d2l

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter


#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# For Progress Bar
from tqdm import tqdm

#ignore warnings
import warnings

warnings.filterwarnings('ignore')

# auto load change
%load_ext autoreload
%autoreload 2

## Improvement
+ use parameter instead of hard-code for base class and task settings
+ metaR `batchnorm1d` is canceled by me

## Dataset set up
+ NELL one: 30 base classes, 21 novel classes(3 way 5 shot 7 sessions)

## Dataset and DataLoader
+ 基于 base class or novel class 数量构建 train_task.
use `next_batch` to get train_task shape: `(4, batch_size, num)`
+ every epoch will get same relations(`self.curr_rel_idx` will ensure this)
### construct support and query
random choose support(相同数量 positive and negative) query(相同数量 positive and negative) according to current relation
+ support query shape: `(batch_size, num, 2(e1_e2), embedding_shape)`

## MetaR train

### embedding entity
get embeddings according to entity id 

### relation learner
![](imgs/img.png)
![](imgs/img_4.png)

### gradient descent in support 
![](imgs/img_1.png)
![](imgs/img_2.png)
![](imgs/img_3.png)

In [5]:
# Test BatchNorm1d
a = torch.ones([3, 1, 2, 100])
inputs.contiguous().view(size[0], size[1], -1)
net = nn.BatchNorm1d(1)
net(a)


ValueError: expected 2D or 3D input (got 4D input)

## eval model

In [32]:
# TODO: simplify it into a function
import json
import numpy as np
tail = ''
data_dir = {
    'train_tasks_in_train': './NELL/train_tasks_in_train.json',
    'train_tasks': './NELL/continual_train_tasks.json',
    'test_tasks': './NELL/test_tasks.json',
    'dev_tasks': './NELL/continual_dev_tasks.json',

    'rel2candidates_in_train': './NELL/rel2candidates_in_train.json',
    'rel2candidates': './NELL/rel2candidates.json',

    'e1rel_e2_in_train': './NELL/e1rel_e2_in_train.json',
    'e1rel_e2': './NELL/e1rel_e2.json',

    'ent2ids': './NELL/ent2ids',
    'ent2vec': './NELL/ent2vec.npy',
}
dataset = dict()
print("loading train_tasks{} ... ...".format(tail))
dataset['train_tasks'] = json.load(open(data_dir['train_tasks' + tail]))
print("loading test_tasks ... ...")
dataset['test_tasks'] = json.load(open(data_dir['test_tasks']))
print("loading dev_tasks ... ...")
dataset['dev_tasks'] = json.load(open(data_dir['dev_tasks']))
print("loading rel2candidates{} ... ...".format(tail))
dataset['rel2candidates'] = json.load(open(data_dir['rel2candidates' + tail]))
print("loading e1rel_e2{} ... ...".format(tail))
dataset['e1rel_e2'] = json.load(open(data_dir['e1rel_e2' + tail]))
print("loading ent2id ... ...")
dataset['ent2id'] = json.load(open(data_dir['ent2ids']))
dataset['ent2emb'] = np.load(data_dir['ent2vec'])
from data_loader import DataLoader
params = {'few': 1, 'num_query': 3, 'batch_size': 16, 'base_classes_few': 10, 'base_classes_num_query': 30, 'base_classes_relation': 30}
train_data_loader = DataLoader(dataset, params, step='train')
dev_data_loader = DataLoader(dataset, params, step='dev')
test_data_loader = DataLoader(dataset, params, step='test')
data_loaders = [train_data_loader, dev_data_loader, test_data_loader]

loading train_tasks ... ...
loading test_tasks ... ...
loading dev_tasks ... ...
loading rel2candidates ... ...
loading e1rel_e2 ... ...
loading ent2id ... ...


In [35]:
for k,v in dataset['dev_tasks'].items():
    print(len(v))

12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12


In [24]:
# 3 support 3 query
dev_tasks = {}
train_tasks = {}
for k, v in dataset['train_tasks'].items():
    dev_tasks[k] = []
    train_tasks[k] = []
    for i in range(len(v)):
        if i < 12:
            dev_tasks[k].append(v.pop())
        else:
            train_tasks[k].append(v.pop())

In [26]:
for k, v in dev_tasks.items():
    print(len(v))

12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12


In [28]:
import json


# Convert and write JSON object to file
with open("NELL/continual_train_tasks", "w") as outfile:
    json.dump(train_tasks, outfile)
with open("NELL/continual_dev_tasks", "w") as outfile:
    json.dump(dev, outfile)
